In [1]:
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
import numpy as np
import threading

In [10]:
sigla = 'BA'
states = gpd.read_file('shapefiles/estadosl_2007.shp')
states = states[states['SIGLAUF3'] == sigla]
xmin, ymin, xmax, ymax = states.bounds.values[0]

width = 50/111 #km/deg
height = 50/111 #km/deg

cols = np.arange(xmin, xmax, width)
rows = np.arange(ymin, ymax, height)[::-1]

data = {}
data['left'] = []
data['top'] = []
data['right'] = []
data['bottom'] = []
data['geometry'] = []

for row in rows:
    row -= rows[1]-rows[0] 
    for col in cols:
        polygon = Polygon([(col, row), (col+width, row), (col+width, row-height), (col, row-height)])
        data['left'].append(col)
        data['top'].append(row)
        data['right'].append(col+width)
        data['bottom'].append(row-height)
        data['geometry'].append(polygon)
        

grid = gpd.GeoDataFrame(data)
grid.crs = {'init': 'epsg:4326'}
grid.to_file("shape/state_%s.shp"%sigla)

In [8]:
class Thread(threading.Thread):
    def __init__(self, polygon, point):
        threading.Thread.__init__(self)
        self.polygon = polygon
        self.point = point
    
    def run(self):
        return self.polygon.contains(self.point)

In [85]:
points = gpd.read_file('shape/flashs/flashs.shp')

# grid = gpd.read_file('shape/state_BA.shp')

# result = {}

# for i, poly in grid.iterrows():
#     result[i] = 0
#     for j, point in points.iterrows():
#         thread = Thread(poly['geometry'], point['geometry'])
#         print(thread.start())